In [1]:
import argparse
import httplib2
import requests
import pandas as pd 

from collections import defaultdict
from dateutil import relativedelta
from googleapiclient.discovery import build
from oauth2client import client
from oauth2client import file
from oauth2client import tools

In [2]:
# Google Cloud Project Client ID & Client Secrets
CLIENT_ID = "165277451909-549djem84js33jh3thql01iccovj6hsl.apps.googleusercontent.com"
CLIENT_SECRET = "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"

# Define Oath scopes with read only access
OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters.readonly"

parser = argparse.ArgumentParser(
    formatter_class=argparse.RawDescriptionHelpFormatter,
    parents=[tools.argparser])
flags = parser.parse_args([])
 
flow = client.OAuth2WebServerFlow(client_id=CLIENT_ID,
                           client_secret=CLIENT_SECRET,
                           scope=OAUTH_SCOPE)
 
storage = file.Storage('searchconsole.dat')
 
credentials = storage.get()
if credentials is None or credentials.invalid:
  credentials = tools.run_flow(flow, storage, flags)
 
# Create an httplib2.Http object and authorize it with our credentials
http = credentials.authorize(http=httplib2.Http())

In [3]:
webmasters_service = build('searchconsole', 'v1', http)
webmasters_service

# Page, Date, Query (JP)

In [4]:
import pandas as pd 

# the website we want to get the data for
websites = [ "https://www.samsung.com/uk/"]
startdate = ['2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01']
enddate = ['2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29']

for property in websites:
    for i in range(len(startdate)):
        all_response = []  # Initialize all_response for each website
        
        startRow = 0

        while (startRow == 0) or (startRow % 25000 == 0):
            request_body = {
                "startDate": startdate[i],
                "endDate": enddate[i],
                "dimensions": ['Date', 'Page', 'Query'],
                "rowLimit": 25000,
                "dataState": "final",
                "startRow": startRow
            }
            response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
            startRow += len(response_data['rows'])
            print("fetched up to " + str(startRow) + " rows of data")

            all_response.extend(response_data['rows'])

        data_for_df = []

        for row in all_response:
            temp = []
            temp.append(row['keys'][0])  # date
            temp.append(row['keys'][1])  #page
            temp.append(row['keys'][2])  #query
            temp.append(row['clicks'])    # clicks
            temp.append(row['impressions'])  # impressions
            temp.append(row['ctr'])       # ctr
            temp.append(row['position'])  # position
            temp.append(property)         # website
            data_for_df.append(temp)

        df = pd.DataFrame(data_for_df, columns=['date', 'page', 'query', 'clicks', 'impressions', 'ctr', 'position', 'website'])
        df.to_csv(f'{property.split("/")[-2]}-{startdate[i]}-date-page-query.csv', index=False)

fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 150000 rows of data
fetched up to 175000 rows of data
fetched up to 200000 rows of data
fetched up to 225000 rows of data
fetched up to 250000 rows of data
fetched up to 275000 rows of data
fetched up to 300000 rows of data
fetched up to 325000 rows of data
fetched up to 350000 rows of data
fetched up to 375000 rows of data
fetched up to 400000 rows of data
fetched up to 425000 rows of data
fetched up to 450000 rows of data
fetched up to 475000 rows of data
fetched up to 500000 rows of data
fetched up to 525000 rows of data
fetched up to 550000 rows of data
fetched up to 575000 rows of data
fetched up to 600000 rows of data
fetched up to 625000 rows of data
fetched up to 650000 rows of data
fetched up to 675000 rows of data
fetched up to 700000 rows of data
fetched up to 725000 rows of data
fetched up to 750

# 당월 데이터

In [4]:
website = ["https://www.samsung.com/uk/"]
STARTDATE = '2024-04-01'
ENDDATE = '2024-04-18'

for property in website:
    all_response = []
    startRow = 0
    while (startRow == 0) or (startRow % 25000 == 0):
        request_body = {
            "startDate" : STARTDATE,
            "endDate" : ENDDATE,
            "dimensions" : ['Date', 'Page', 'Query'],
            "rowLimit" : 25000,
            "dateState" : "final",
            "startRow" : startRow
        }
        response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
        startRow += len(response_data['rows'])
        print("fetched up to " + str(startRow) + " rows of data")
        all_response.extend(response_data['rows'])
    
    data_for_df = []
    
    for row in all_response:
        temp = []
        temp.append(row['keys'][0])  #date
        temp.append(row['keys'][1])  #page
        temp.append(row['keys'][2])  #query
        temp.append(row['clicks'])
        temp.append(row['impressions'])
        temp.append(row['ctr'])
        temp.append(row['position'])
        temp.append(property)
        data_for_df.append(temp)
    
    df = pd.DataFrame(data_for_df, columns = ['date', 'page', 'query', 'clicks', 'impressions', 'ctr', 'position', 'website'])
    df.to_csv(f'{property.split("/")[-2]}-{STARTDATE}-to-{df['date'].max()}-date-page.csv', index=False)

fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 150000 rows of data
fetched up to 175000 rows of data
fetched up to 200000 rows of data
fetched up to 225000 rows of data
fetched up to 250000 rows of data
fetched up to 275000 rows of data
fetched up to 300000 rows of data
fetched up to 325000 rows of data
fetched up to 350000 rows of data
fetched up to 375000 rows of data
fetched up to 400000 rows of data
fetched up to 425000 rows of data
fetched up to 450000 rows of data
fetched up to 475000 rows of data
fetched up to 500000 rows of data
fetched up to 525000 rows of data
fetched up to 538282 rows of data


# 특정페이지

In [7]:
website = ["https://www.samsung.com/uk/"]
STARTDATE = '2024-04-01'
ENDDATE = '2024-04-18'

for property in website:
    all_response = []
    startRow = 0
    while (startRow == 0) or (startRow % 25000 == 0):
        request_body = {
            "startDate" : STARTDATE,
            "endDate" : ENDDATE,
            "dimensions" : ['Date', 'Page', 'Query'],
            "dimensionFilterGroups" : [{
                "filters" : [{
                    "dimension" : 'page',
                    "expression" : "https://www.samsung.com/uk/apps/samsung-health/"
                }]
            }],
            "rowLimit" : 25000,
            "dateState" : "final",
            "startRow" : startRow
        }
        response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
        startRow += len(response_data['rows'])
        print("fetched up to " + str(startRow) + " rows of data")
        all_response.extend(response_data['rows'])
    
    data_for_df = []
    
    for row in all_response:
        temp = []
        temp.append(row['keys'][0])  #date
        temp.append(row['keys'][1])  #page
        temp.append(row['keys'][2])  #query
        temp.append(row['clicks'])
        temp.append(row['impressions'])
        temp.append(row['ctr'])
        temp.append(row['position'])
        temp.append(property)
        data_for_df.append(temp)
    
    df = pd.DataFrame(data_for_df, columns = ['date', 'page', 'query', 'clicks', 'impressions', 'ctr', 'position', 'website'])
    df.to_csv('page-filter-date-page-query.csv', index=False)

fetched up to 143 rows of data


# 전국가사이트에 buying guide 존재 여부 확인

In [4]:
site_list = webmasters_service.sites().list().execute()

website = []
for each in site_list['siteEntry']:
    website.append(each['siteUrl'])

website

['https://www.samsung.com/cn/',
 'https://www.samsung.com/sec/',
 'https://www.samsung.com/hk/',
 'https://www.samsung.com/sg/',
 'https://www.samsung.com/uk/',
 'https://www.samsung.com/bg/',
 'https://www.samsung.com/iq_ku/',
 'https://www.samsung.com/id/',
 'https://www.samsung.com/ie/',
 'https://www.samsung.com/ua/',
 'https://www.samsung.com/us/smartphones/galaxy-s24/galaxy-ai/',
 'https://www.samsung.com/kz_ru/',
 'https://www.samsung.com/ph/',
 'https://www.samsung.com/il/',
 'https://www.samsung.com/ch/',
 'https://www.samsung.com/fr/',
 'https://www.samsung.com/eg/',
 'https://www.samsung.com/si/',
 'https://www.samsung.com/lb/',
 'https://www.samsung.com/my/',
 'https://www.samsung.com/levant/',
 'https://www.samsung.com/za/',
 'https://www.samsung.com/lv/',
 'https://www.ascentkorea.com/',
 'https://www.samsung.com/ca/',
 'https://www.samsung.com/pl/',
 'https://www.samsung.com/africa_en/',
 'https://www.samsung.com/dk/',
 'https://www.samsung.com/levant_ar/',
 'https://www

In [7]:
urls_to_remove = ['https://www.samsung.com/us/smartphones/galaxy-s24/galaxy-ai/',
                  'https://www.samsung.com/',
                  'https://www.ascentkorea.com/']

for url in urls_to_remove:
    if url in website:
        website.remove(url)



['https://www.samsung.com/cn/',
 'https://www.samsung.com/sec/',
 'https://www.samsung.com/hk/',
 'https://www.samsung.com/sg/',
 'https://www.samsung.com/uk/',
 'https://www.samsung.com/bg/',
 'https://www.samsung.com/iq_ku/',
 'https://www.samsung.com/id/',
 'https://www.samsung.com/ie/',
 'https://www.samsung.com/ua/',
 'https://www.samsung.com/kz_ru/',
 'https://www.samsung.com/ph/',
 'https://www.samsung.com/il/',
 'https://www.samsung.com/ch/',
 'https://www.samsung.com/fr/',
 'https://www.samsung.com/eg/',
 'https://www.samsung.com/si/',
 'https://www.samsung.com/lb/',
 'https://www.samsung.com/my/',
 'https://www.samsung.com/levant/',
 'https://www.samsung.com/za/',
 'https://www.samsung.com/lv/',
 'https://www.samsung.com/ca/',
 'https://www.samsung.com/pl/',
 'https://www.samsung.com/africa_en/',
 'https://www.samsung.com/dk/',
 'https://www.samsung.com/levant_ar/',
 'https://www.samsung.com/be/',
 'https://www.samsung.com/gr/',
 'https://www.samsung.com/co/',
 'https://www.s

In [8]:
len(website)

93

In [23]:
STARTDATE = '2023-01-01'
ENDDATE = '2024-04-28'

for property in website:
    all_response = []
    startRow = 0
    while (startRow == 0) or (startRow % 25000 == 0):
        request_body = {
            "startDate" : STARTDATE,
            "endDate" : ENDDATE,
            "dimensions" : ['Page'],
            "dimensionFilterGroups" : [{
                "filters" : [{
                    "dimension" : 'page',
                    "operator" : 'contains',
                    "expression" : 'buying-guide'
                }]
            }],
            "rowLimit" : 25000,
            "dateState" : "final",
            "startRow" : startRow
        }
        response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
        startRow += len(response_data['rows'])
        print("fetched up to " + property.split('/')[-2] + str(startRow) + " rows of data")
        all_response.extend(response_data['rows'])
    
    data_for_df = []
    
    for row in all_response:
        temp = []
        #temp.append(row['keys'][0])  #date
        temp.append(row['keys'][0])  #page
        temp.append(row['clicks'])
        temp.append(row['impressions'])
        temp.append(row['ctr'])
        temp.append(row['position'])
        temp.append(property)
        data_for_df.append(temp)
    
    df = pd.DataFrame(data_for_df, columns = ['page', 'clicks', 'impressions', 'ctr', 'position', 'website'])
    df.to_csv('buying-guide-page.csv', index=False)

fetched up to cn11 rows of data
fetched up to sec12 rows of data
fetched up to hk19 rows of data
fetched up to sg35 rows of data
fetched up to uk127 rows of data
fetched up to bg11 rows of data
fetched up to iq_ku22 rows of data
fetched up to id30 rows of data
fetched up to ie12 rows of data
fetched up to ua54 rows of data
fetched up to kz_ru14 rows of data
fetched up to ph42 rows of data
fetched up to il19 rows of data
fetched up to ch16 rows of data
fetched up to fr37 rows of data
fetched up to eg11 rows of data
fetched up to si21 rows of data
fetched up to lb15 rows of data
fetched up to my35 rows of data
fetched up to levant22 rows of data
fetched up to za30 rows of data
fetched up to lv11 rows of data
fetched up to ca19 rows of data
fetched up to pl137 rows of data
fetched up to africa_en29 rows of data
fetched up to dk29 rows of data
fetched up to levant_ar21 rows of data
fetched up to be33 rows of data
fetched up to gr22 rows of data
fetched up to co27 rows of data
fetched up to

KeyError: 'rows'

In [24]:
all_response

[]

In [31]:
import pandas as pd

STARTDATE = '2023-01-01'
ENDDATE = '2024-04-28'

all_response = []  # 모든 웹사이트의 데이터를 쌓을 리스트
data_for_df = []
for property in website:
    startRow = 0
    while (startRow == 0) or (startRow % 25000 == 0):
        request_body = {
            "startDate" : STARTDATE,
            "endDate" : ENDDATE,
            "dimensions" : ['Page'],
            "dimensionFilterGroups" : [{
                "filters" : [{
                    "dimension" : 'page',
                    "operator" : 'contains',
                    "expression" : 'buying-guide'
                }]
            }],
            "rowLimit" : 25000,
            "dateState" : "final",
            "startRow" : startRow
        }
        try:
            response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
            if 'rows' in response_data:
                startRow += len(response_data['rows'])
                print("fetched up to " + property.split('/')[-2] + str(startRow) + " rows of data")
                all_response.extend(response_data['rows'])
                for row in response_data['rows']:  # 각 웹사이트의 데이터를 data_for_df에 추가
                    temp = []
                    temp.append(row['keys'][0])  # page
                    temp.append(row['clicks'])
                    temp.append(row['impressions'])
                    temp.append(row['ctr'])
                    temp.append(row['position'])
                    temp.append(property)
                    data_for_df.append(temp)
            else:
                break
        except KeyError:
            print("KeyError: 'rows' not found in API response")
            break
    
df = pd.DataFrame(data_for_df, columns=['page', 'clicks', 'impressions', 'ctr', 'position', 'website'])
df.to_csv('buying-guide-page.csv', index=False)


fetched up to cn11 rows of data
fetched up to sec12 rows of data
fetched up to hk19 rows of data
fetched up to sg35 rows of data
fetched up to uk127 rows of data
fetched up to bg11 rows of data
fetched up to iq_ku22 rows of data
fetched up to id30 rows of data
fetched up to ie12 rows of data
fetched up to ua54 rows of data
fetched up to kz_ru14 rows of data
fetched up to ph42 rows of data
fetched up to il19 rows of data
fetched up to ch16 rows of data
fetched up to fr37 rows of data
fetched up to eg11 rows of data
fetched up to si21 rows of data
fetched up to lb15 rows of data
fetched up to my35 rows of data
fetched up to levant22 rows of data
fetched up to za30 rows of data
fetched up to lv11 rows of data
fetched up to ca19 rows of data
fetched up to pl137 rows of data
fetched up to africa_en29 rows of data
fetched up to dk29 rows of data
fetched up to levant_ar21 rows of data
fetched up to be33 rows of data
fetched up to gr22 rows of data
fetched up to co27 rows of data
fetched up to

# 특정페이지 데이터 전체국가에서 불러오기 (date,pate)

In [6]:
site_list = webmasters_service.sites().list().execute()

website = []
for each in site_list['siteEntry']:
    website.append(each['siteUrl'])
    
urls_to_remove = ['https://www.samsung.com/us/smartphones/galaxy-s24/galaxy-ai/',
                  'https://www.samsung.com/',
                  'https://www.ascentkorea.com/']

for url in urls_to_remove:
    if url in website:
        website.remove(url)

len(website)

93

In [11]:
import pandas as pd

STARTDATE = '2024-01-01'
ENDDATE = '2024-04-28'

all_response = []  # 모든 웹사이트의 데이터를 쌓을 리스트
data_for_df = []
for property in website:
    startRow = 0
    while (startRow == 0) or (startRow % 25000 == 0):
        request_body = {
            "startDate" : STARTDATE,
            "endDate" : ENDDATE,
            "dimensions" : ['Page'],
            "dimensionFilterGroups" : [{
                "filters" : [{
                    "dimension" : 'page',
                    "operator" : 'contains',
                    "expression" : 'explore'
                }]
            }],
            "rowLimit" : 25000,
            "dateState" : "final",
            "startRow" : startRow
        }
        try:
            response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
            if 'rows' in response_data:
                startRow += len(response_data['rows'])
                print("fetched up to " + property.split('/')[-2] + str(startRow) + " rows of data")
                all_response.extend(response_data['rows'])
                for row in response_data['rows']:  # 각 웹사이트의 데이터를 data_for_df에 추가
                    temp = []
                    #temp.append(row['keys'][0])  # date
                    temp.append(row['keys'][0])  # page
                    temp.append(row['clicks'])
                    temp.append(row['impressions'])
                    temp.append(row['ctr'])
                    temp.append(row['position'])
                    temp.append(property)
                    data_for_df.append(temp)
            else:
                break
        except KeyError:
            print("KeyError: 'rows' not found in API response")
            break
    
df = pd.DataFrame(data_for_df, columns=['page', 'clicks', 'impressions', 'ctr', 'position', 'website'])
df.to_csv('explore-page.csv', index=False)


fetched up to ca23 rows of data
fetched up to africa_fr85 rows of data
fetched up to za69 rows of data
fetched up to ps65 rows of data
fetched up to latin69 rows of data
fetched up to ae67 rows of data
fetched up to dk180 rows of data
fetched up to pt116 rows of data
fetched up to ar102 rows of data
fetched up to eg61 rows of data
fetched up to iq_ar48 rows of data
fetched up to se185 rows of data
fetched up to si111 rows of data
fetched up to in54 rows of data
fetched up to ba111 rows of data
fetched up to de181 rows of data
fetched up to bd65 rows of data
fetched up to us95 rows of data
fetched up to africa_en98 rows of data
fetched up to pe80 rows of data
fetched up to ru306 rows of data
fetched up to pk61 rows of data
fetched up to sa124 rows of data
fetched up to mx85 rows of data
fetched up to ie84 rows of data
fetched up to ch150 rows of data
fetched up to pl268 rows of data
fetched up to kz_ru105 rows of data
fetched up to www.samsung.com.cn59 rows of data
fetched up to il132 r

# 전체국가의 1년치 클릭수 수집

In [35]:
site_list = webmasters_service.sites().list().execute()

website = []
for each in site_list['siteEntry']:
    website.append(each['siteUrl'])
    
urls_to_remove = ['https://www.samsung.com/us/smartphones/galaxy-s24/galaxy-ai/',
                  'https://www.samsung.com/',
                  'https://www.ascentkorea.com/',
                  'https://www.samsung.com.cn/']

for url in urls_to_remove:
    if url in website:
        website.remove(url)

len(website)

92

In [34]:
import pandas as pd

STARTDATE = '2024-01-01'
ENDDATE = '2024-03-31'

all_response = []  # 모든 웹사이트의 데이터를 쌓을 리스트
data_for_df = []
for property in website:
    startRow = 0
    while (startRow == 0) or (startRow % 25000 == 0):
        request_body = {
            "startDate" : STARTDATE,
            "endDate" : ENDDATE,
            "rowLimit" : 25000,
            "dateState" : "final",
            "startRow" : startRow
        }
        try:
            response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
            if 'rows' in response_data:
                startRow += len(response_data['rows'])
                print("fetched up to " + property.split('/')[-2] + str(startRow) + " rows of data")
                all_response.extend(response_data['rows'])
                for row in response_data['rows']:  # 각 웹사이트의 데이터를 data_for_df에 추가
                    temp = []
                    temp.append(row['keys'][0])  # date
                    temp.append(row['keys'][1])  # page
                    temp.append(row['clicks'])
                    temp.append(row['impressions'])
                    temp.append(row['ctr'])
                    temp.append(row['position'])
                    temp.append(property)
                    data_for_df.append(temp)
            else:
                break
        except KeyError:
            print("KeyError: 'rows' not found in API response")
            break
    
df = pd.DataFrame(data_for_df, columns=['date', 'page', 'clicks', 'impressions', 'ctr', 'position', 'website'])
df.to_csv('buying-guide-date-page.csv', index=False)


fetched up to uz_uz5197 rows of data
fetched up to levant4736 rows of data
fetched up to my7965 rows of data
fetched up to lb3844 rows of data
fetched up to eg2278 rows of data
fetched up to si7768 rows of data
fetched up to il5977 rows of data
fetched up to fr7313 rows of data
fetched up to ph9217 rows of data
fetched up to kz_ru6539 rows of data
fetched up to ch5240 rows of data
fetched up to ie769 rows of data
fetched up to id7678 rows of data
fetched up to ua12200 rows of data
fetched up to bg4969 rows of data
fetched up to iq_ku5303 rows of data
fetched up to hk6520 rows of data
fetched up to sg7363 rows of data
fetched up to cn3430 rows of data
fetched up to sec5421 rows of data
fetched up to ba7238 rows of data
fetched up to mk6711 rows of data
fetched up to ps1966 rows of data
fetched up to latin_en984 rows of data
fetched up to n_africa5249 rows of data
fetched up to ae_ar6759 rows of data
fetched up to pk5467 rows of data
fetched up to no6853 rows of data
fetched up to uy4792

## 완성본

In [91]:
site_list = webmasters_service.sites().list().execute()

websites = []
for each in site_list['siteEntry']:
    websites.append(each['siteUrl'])
    
urls_to_remove = ['https://www.samsung.com/us/smartphones/galaxy-s24/galaxy-ai/',
                  'https://www.samsung.com/',
                  'https://www.ascentkorea.com/',
                  'https://www.samsung.com.cn/']

for url in urls_to_remove:
    if url in websites:
        websites.remove(url)

len(websites)

92

In [94]:
import pandas as pd

STARTDATE = ['2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01']
ENDDATE = ['2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29', '2024-03-31']


data_for_df = []

for property in websites:
    startRow = 0
    for_counting = []
    for i in range(len(STARTDATE)):
        request_body = {
                    "startDate" : STARTDATE[i],
                    "endDate" : ENDDATE[i],
                    "aggregationType" : 'byPage'
                }
        try:
            response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
            if 'rows' in response_data:
                temp = []
                temp.append(STARTDATE[i])
                temp.append(response_data['rows'][0]['clicks'])
                temp.append(response_data['rows'][0]['impressions'])
                temp.append(response_data['rows'][0]['ctr'])
                temp.append(response_data['rows'][0]['position'])
                temp.append(property)
                data_for_df.append(temp)
                for_counting.append(temp)
            else:
                break
        except KeyError:
            print("KeyError: 'rows' not found in API response")
            break

    startRow += len(for_counting)
    print(property.split('/')[-2], startRow)

df = pd.DataFrame(data_for_df, columns = ['month', 'clicks', 'impressions', 'ctr', 'position', 'website'])
df.to_csv('all-site-monthly.csv', index=False)
                    
        

ch_fr 15
gr 15
ae_ar 15
n_africa 15
kz_ru 15
nz 15
py 15
rs 15
us 15
lb 15
br 15
az 15
th 15
ie 15
es 15
ps 15
uz_uz 15
se 15
ee 15
cl 15
dk 15
ca_fr 15
pk 15
za 15
mn 0
tr 15
il 15
fi 15
ru 15
cz 15
in 15
sk 15
ba 15
bd 15
sa_en 15
sa 15
ar 15
al 15
mk 15
pt 15
ph 15
hk_en 15
eg 15
hr 15
hk 15
levant 15
vn 15
mx 15
fr 15
kz_kz 15
ca 15
no 15
latin_en 15
sec 15
iran 15
uz_ru 15
pe 15
pl 15
hu 15
id 15
mm 15
au 15
iq_ar 15
ch 15
lv 15
de 15
tw 15
si 15
ae 15
iq_ku 15
be_fr 15
jp 0
lt 15
africa_fr 15
my 15
bg 15
ge 0
ua 15
africa_pt 15
be 15
uy 15
ro 15
cn 15
sg 15
co 15
africa_en 15
it 15
latin 15
at 15
uk 15
nl 15
levant_ar 15


## jp처럼 제대로 수집안되는 경우 test

In [4]:
import pandas as pd

websites = ['https://www.samsung.com.cn']
STARTDATE = ['2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01']
ENDDATE = ['2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29', '2024-03-31']

data_for_df = []

for property in websites:
    startRow = 0
    for_counting = []  # Initialize for_counting list here
    for i in range(len(STARTDATE)):
        request_body = {
                    "startDate" : STARTDATE[i],
                    "endDate" : ENDDATE[i],
                    "aggregationType" : 'byPage'
                }
        try:
            response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
            if 'rows' in response_data:
                temp = []
                temp.append(STARTDATE[i])
                temp.append(response_data['rows'][0]['clicks'])
                temp.append(response_data['rows'][0]['impressions'])
                temp.append(response_data['rows'][0]['ctr'])
                temp.append(response_data['rows'][0]['position'])
                temp.append(property)
                data_for_df.append(temp)
                for_counting.append(temp)
            else:
                # If there are no rows in the response data, append a placeholder list to for_counting
                for_counting.append([STARTDATE[i], 0, 0, 0, 0, property])
        except KeyError:
            print("KeyError: 'rows' not found in API response")
            break

    startRow += len(for_counting)
    print(property.split('/')[-2], startRow)

df = pd.DataFrame(data_for_df, columns = ['month', 'clicks', 'impressions', 'ctr', 'position', 'website'])
df.to_csv('all-site-monthly-cn.csv', index=False)


 15


In [49]:
import pandas as pd
websites = ["https://www.samsung.com/uk/", "https://www.samsung.com/in/"]
STARTDATE = ['2024-02-01', '2024-03-01']
ENDDATE = ['2024-02-29', '2024-03-31']


data_for_df = []
for property in websites:
    for i in range(len(STARTDATE)):
        request_body = {
                    "startDate" : STARTDATE,
                    "endDate" : ENDDATE,
                    "aggregationType" : 'byPage'
                }
        response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
        print(response_data)

{'rows': [{'clicks': 3756691, 'impressions': 143827125, 'ctr': 0.026119488935067013, 'position': 12.262415208535941}], 'responseAggregationType': 'byPage'}
{'rows': [{'clicks': 3756691, 'impressions': 143827125, 'ctr': 0.026119488935067013, 'position': 12.262415208535941}], 'responseAggregationType': 'byPage'}
{'rows': [{'clicks': 3515842, 'impressions': 222751638, 'ctr': 0.01578368640323983, 'position': 6.971126690435381}], 'responseAggregationType': 'byPage'}
{'rows': [{'clicks': 3515842, 'impressions': 222751638, 'ctr': 0.01578368640323983, 'position': 6.971126690435381}], 'responseAggregationType': 'byPage'}


In [57]:
response_data

{'rows': [{'clicks': 3515842,
   'impressions': 222751638,
   'ctr': 0.01578368640323983,
   'position': 6.971126690435381}],
 'responseAggregationType': 'byPage'}

In [58]:
response_data['rows']

[{'clicks': 3515842,
  'impressions': 222751638,
  'ctr': 0.01578368640323983,
  'position': 6.971126690435381}]

In [64]:
response_data['rows']['clicks']

TypeError: list indices must be integers or slices, not str

In [63]:
response_data['rows'][0]

{'clicks': 3515842,
 'impressions': 222751638,
 'ctr': 0.01578368640323983,
 'position': 6.971126690435381}

In [60]:
response_data['rows'][0]['clicks']

3515842

In [67]:
response_data['rows'][1]

IndexError: list index out of range

In [62]:
import pandas as pd
websites = ["https://www.samsung.com/uk/", "https://www.samsung.com/in/"]
STARTDATE = ['2024-02-01', '2024-03-01']
ENDDATE = ['2024-02-29', '2024-03-31']

all_response = [] #얘가 여기에 있으면 all response가 2개만 나옴
for property in websites:
    for i in range(len(STARTDATE)):
        request_body = {
                    "startDate" : STARTDATE,
                    "endDate" : ENDDATE,
                    "aggregationType" : 'byPage'
                }
        response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()
        all_response.extend(response_data['rows'])

print(all_response)

[{'clicks': 3756691, 'impressions': 143827125, 'ctr': 0.026119488935067013, 'position': 12.262415208535941}, {'clicks': 3756691, 'impressions': 143827125, 'ctr': 0.026119488935067013, 'position': 12.262415208535941}, {'clicks': 3515842, 'impressions': 222751638, 'ctr': 0.01578368640323983, 'position': 6.971126690435381}, {'clicks': 3515842, 'impressions': 222751638, 'ctr': 0.01578368640323983, 'position': 6.971126690435381}]


In [84]:
import pandas as pd

websites = ["https://www.samsung.com/uk/", "https://www.samsung.com/in/"]
STARTDATE = ['2024-02-01', '2024-03-01']
ENDDATE = ['2024-02-29', '2024-03-31']

data_for_df = []

for property in websites:
    startRow = 0
    for_counting = []
    for i in range(len(STARTDATE)):
        request_body = {
                    "startDate" : STARTDATE[i],
                    "endDate" : ENDDATE[i],
                    "aggregationType" : 'byPage'
                }
        response_data = webmasters_service.searchanalytics().query(siteUrl=property, body=request_body).execute()

        temp = []
        temp.append(STARTDATE[i])
        temp.append(response_data['rows'][0]['clicks'])
        temp.append(response_data['rows'][0]['impressions'])
        temp.append(response_data['rows'][0]['ctr'])
        temp.append(response_data['rows'][0]['position'])
        temp.append(property)
        data_for_df.append(temp)
        for_counting.append(temp)

        print(for_counting)
    startRow += len(for_counting)
    print(property.split('/')[-2], startRow)

#df = pd.DataFrame(data_for_df, columns = ['month', 'clicks', 'impressions', 'ctr', 'position', 'website'])
#df.to_csv('all-site-monthly.csv', index=False)
                    

[['2024-02-01', 3756691, 143827125, 0.026119488935067013, 12.262415208535941, 'https://www.samsung.com/uk/']]
[['2024-02-01', 3756691, 143827125, 0.026119488935067013, 12.262415208535941, 'https://www.samsung.com/uk/'], ['2024-03-01', 4032790, 153992937, 0.026188149135697048, 12.458601922762211, 'https://www.samsung.com/uk/']]
uk 2
[['2024-02-01', 3515842, 222751638, 0.01578368640323983, 6.971126690435381, 'https://www.samsung.com/in/']]
[['2024-02-01', 3515842, 222751638, 0.01578368640323983, 6.971126690435381, 'https://www.samsung.com/in/'], ['2024-03-01', 4336876, 260025345, 0.016678666458456195, 6.911792802351632, 'https://www.samsung.com/in/']]
in 2
